## 1.1 手写实现 DSA

In [ ]:
class SHA2_256:
    @staticmethod
    def _rr(x, n): return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

    def sha256(self, msg):
        h = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
             0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]
        K = [0x428a2f98,0x71374491,0xb5c0fbcf,0xe9b5dba5,0x3956c25b,0x59f111f1,0x923f82a4,0xab1c5ed5,
             0xd807aa98,0x12835b01,0x243185be,0x550c7dc3,0x72be5d74,0x80deb1fe,0x9bdc06a7,0xc19bf174,
             0xe49b69c1,0xefbe4786,0x0fc19dc6,0x240ca1cc,0x2de92c6f,0x4a7484aa,0x5cb0a9dc,0x76f988da,
             0x983e5152,0xa831c66d,0xb00327c8,0xbf597fc7,0xc6e00bf3,0xd5a79147,0x06ca6351,0x14292967,
             0x27b70a85,0x2e1b2138,0x4d2c6dfc,0x53380d13,0x650a7354,0x766a0abb,0x81c2c92e,0x92722c85,
             0xa2bfe8a1,0xa81a664b,0xc24b8b70,0xc76c51a3,0xd192e819,0xd6990624,0xf40e3585,0x106aa070,
             0x19a4c116,0x1e376c08,0x2748774c,0x34b0bcb5,0x391c0cb3,0x4ed8aa4a,0x5b9cca4f,0x682e6ff3,
             0x748f82ee,0x78a5636f,0x84c87814,0x8cc70208,0x90befffa,0xa4506ceb,0xbef9a3f7,0xc67178f2]
        msg = bytes.fromhex(msg)
        bl = len(msg) * 8
        msg += b'\x80'
        while (len(msg) * 8) % 512 != 448: msg += b'\x00'
        msg += bl.to_bytes(8, 'big')
        for i in range(0, len(msg), 64):
            chunk = msg[i:i+64]
            w = [int.from_bytes(chunk[j:j+4], 'big') for j in range(0, 64, 4)]
            for j in range(16, 64):
                s0 = (self._rr(w[j-15],7)^self._rr(w[j-15],18)^(w[j-15]>>3))
                s1 = (self._rr(w[j-2],17)^self._rr(w[j-2],19)^(w[j-2]>>10))
                w.append((w[j-16]+s0+w[j-7]+s1)&0xFFFFFFFF)
            a,b,c,d,e,f,g,h = h
            for j in range(64):
                S1 = self._rr(e,6)^self._rr(e,11)^self._rr(e,25)
                ch = (e&f)^((~e)&g)
                t1 = (h+S1+ch+K[j]+w[j])&0xFFFFFFFF
                S0 = self._rr(a,2)^self._rr(a,13)^self._rr(a,22)
                maj = (a&b)^(a&c)^(b&c)
                t2 = (S0+maj)&0xFFFFFFFF
                h,g,f,e,d,c,b,a = g,f,e,(d+t1)&0xFFFFFFFF,c,b,a,(t1+t2)&0xFFFFFFFF
            h = [(h[i]+locals()[x])&0xFFFFFFFF for i,x in enumerate('abcdefgh')]
        return (h[0]<<224)|(h[1]<<192)|(h[2]<<160)|(h[3]<<128)|(h[4]<<96)|(h[5]<<64)|(h[6]<<32)|h[7]

In [32]:
import random
# import hashlib  # 仅使用SHA256函数，为了简化代码，实际上，实验五中已经实现了SHA256函数，可以直接使用（注意需要把返回的字符串转为整数）
import math  # 仅使用 gcd 函数，为了简化代码，实际上，实验五中已经实现了 gcd 函数，可以直接使用

class DSA:
    def __init__(self, p, q, g, x = None):
        self.p = p
        self.q = q
        self.g = g

        if x is None:
            x = random.randint(1, self.q - 1)

        self.private_key = x
        self.public_key = pow(self.g, x, self.p)
    
    def get_k_randomly(self):  
        """生成一个小于 q 且与 q 互素的随机整数 k"""
        val = random.randint(1, self.q - 1)
        while math.gcd(val, self.q) != 1:
            val = random.randint(1, self.q - 1)
        return val

    def sign(self, message: str):# (..., k: int = None):  # 不能将 k 作为参数输入，这里仅为了测试使用
        # if k is None:
            # k = self.get_k_randomly()
        k = self.get_k_randomly()
        h = self.hash_int(message)
        r = pow(self.g, k, self.p) % self.q
        s = (pow(k, -1, self.q) * (h + self.private_key * r)) % self.q
        
        if r == 0 or s == 0:
            return self.sign(message, k)
            
        return r, s
    
    def hash_int(self, message: str) -> int:
        """将消息哈希为整数"""
        Sha256 = SHA2_256()
        digest = Sha256.sha256(message.encode('utf-8').hex())
        return digest % self.q
        # digest = hashlib.sha256(message.encode()).digest()
        # return int.from_bytes(digest, 'big') % self.q
    
    def verify(self, message: str, r: int, s: int) -> bool:
        if not (1 <= r <= self.q - 1 and 1 <= s <= self.q - 1):
            return False
            
        h = self.hash_int(message)
        inv_s = pow(s, -1, self.q)
        u1 = (h * inv_s) % self.q
        u2 = (r * inv_s) % self.q
        
        v = ((pow(self.g, u1, self.p) * pow(self.public_key, u2, self.p)) % self.p) % self.q  # 这里需要先 mod p 再 mod q
        
        return v == r
    
# def get_k(q):  # 这段代码按照道理应该放在 DSA 类里面（会出现逆向破解 k 值的风险），但为了方便测试，就把它单独提出来
    # """生成一个小于 q 且与 q 互素的随机整数 k"""
    # val = random.randint(1, q - 1)
    # while math.gcd(val, q) != 1:
        # val = random.randint(1, q - 1)
    # return val

In [33]:
p = 0x87A8E61DB4B6663CFFBBD19C651959998CEEF608660DD0F25D2CEED4435E3B00E00DF8F1D61957D4FAF7DF4561B2AA3016C3D91134096FAA3BF4296D830E9A7C209E0C6497517ABD5A8A9D306BCF67ED91F9E6725B4758C022E0B1EF4275BF7B6C5BFC11D45F9088B941F54EB1E59BB8BC39A0BF12307F5C4FDB70C581B23F76B63ACAE1CAA6B7902D52526735488A0EF13C6D9A51BFA4AB3AD8347796524D8EF6A167B5A41825D967E144E5140564251CCACB83E6B486F6B3CA3F7971506026C0B857F689962856DED4010ABD0BE621C3A3960A54E710C375F26375D7014103A4B54330C198AF126116D2276E11715F693877FAD7EF09CADB094AE91E1A1597
g = 0x3FB32C9B73134D0B2E77506660EDBD484CA7B18F21EF205407F4793A1A0BA12510DBC15077BE463FFF4FED4AAC0BB555BE3A6C1B0C6B47B1BC3773BF7E8C6F62901228F8C28CBB18A55AE31341000A650196F931C77A57F2DDF463E5E9EC144B777DE62AAAB8A8628AC376D282D6ED3864E67982428EBC831D14348F6F2F9193B5045AF2767164E1DFC967C1FB3F2E55A4BD1BFFE83B9C80D052B985D182EA0ADB2A3B7313D3FE14C8484B1E052588B9B7D2BBD2DF016199ECD06E1557CD0915B3353BBB64E0EC377FD028370DF92B52C7891428CDC67EB6184B523D1DB246C32F63078490F00EF8D647D148D47954515E2327CFEF98C582664B4C0F6CC41659
q = 0x8CF83642A709A097B447997640129DA299B1A47D1EB3750BA308B0FE64F5FBD3
message = "Hello, DSA!"

x = random.randint(1, q - 1)
# k = get_k(q)   # 这段代码按照道理应该放在 DSA 类里面，不应该单独提取出来（会出现逆向破解 k 值的风险），但为了方便测试，就把它单独提出来
print(f"私钥: {hex(x)}")
# print(f"k: {hex(k)}")

私钥: 0x55da638a536ac270f1527978305c9c96dc24bc0866f89dab72b21f3197090525


In [37]:
dsa = DSA(p, q, g, x)  
# dsa = DSA(p, q, g, x, k)  # 不应该出现 k 值输入，仅测试时使用（可以将本行的 k 删除，并不会影响代码正常工作，因为代码已经进行随机化 k 值生成）
r, s = dsa.sign(message)
print(f"手写签名结果: ")
print(f"r = {hex(r)}")
print(f"s = {hex(s)}")
if dsa.verify(message, r, s):
    print("签名验证通过")
else:
    print("签名验证失败")

手写签名结果: 
r = 0x668e78925ce80285d7e04285ddec75767d4b9f4a48e7338535410437613fd78d
s = 0x2b4f4b2e326156ddd8fd354b51ece734843118554d76440775adce324b719495
签名验证通过


**k 值应该仅在签名时函数内部使用，不应该在其他任何地方出现（本实验仅仅为了方便测试才将 k 值拿出进行固定测试**

## 1.2 标准库实现 DSA

In [13]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import hashes
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives.asymmetric.dsa import DSAPrivateNumbers

def DSA_std(p, q, g, message, private_key_x = None):
    params_numbers = dsa.DSAParameterNumbers(p, q, g)
    parameters = params_numbers.parameters(backend=default_backend())

    if private_key_x is None:
        private_key = parameters.generate_private_key()
        public_key = private_key.public_key()
    else:
        public_key_y = pow(g, private_key_x, p)  
        dsa_private_numbers = DSAPrivateNumbers(
            x=private_key_x,
            public_numbers=dsa.DSAPublicNumbers(
                y=public_key_y,
                parameter_numbers=params_numbers
            )
        )
        private_key = dsa_private_numbers.private_key(backend=default_backend())
        public_key = private_key.public_key()

    message_bytes = message.encode('utf-8')

    signature = private_key.sign(
        message_bytes,
        hashes.SHA256()
    )

    return signature, public_key, message_bytes  

def extract_rs_from_signature(signature_bytes, q_length=20):
    idx = 0  
    idx += 2

    r_len = signature_bytes[idx + 1]
    r_bytes = signature_bytes[idx + 2 : idx + 2 + r_len]
    r = int.from_bytes(r_bytes, 'big')
    idx += 2 + r_len

    s_len = signature_bytes[idx + 1]
    s_bytes = signature_bytes[idx + 2 : idx + 2 + s_len]
    s = int.from_bytes(s_bytes, 'big')
    return r, s

In [16]:
signature, public_key, message_bytes = DSA_std(p, q, g, message, x)
r, s = extract_rs_from_signature(signature)

print(f"标准库签名结果:")
print(f"r = {hex(r)}")
print(f"s = {hex(s)}")
    
try:
    public_key.verify(
        signature,
        message_bytes,
        hashes.SHA256()
    )
    print("签名验证通过！")
except InvalidSignature:
    print("签名验证失败！")

标准库签名结果:
r = 0x8b65339fa87766e9f7d85a9fff7e982482b7b5b5b79a8b85015fababf1d267ad
s = 0x81bfca37148a958096200d87dbd3d15dd00c5a2a9ce7dc49b676dd78a39cc136
签名验证通过！


<span style='color:red'>标准库不支持固定 k 值，只能生成随机签名，无法进行签名固定<span>

## 2.1 手写实现 ECDSA

In [44]:
p = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF
a = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFC
b = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B
Gx = 0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296
Gy = 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5
n = 0xFFFFFFFF00000000FFFFFFFFFFFFFFFFBCE6FAADA7179E84F3B9CAC2FC632551
G = (Gx, Gy)
INF = (0, 1, 0)

def mod_pow(x, y, m):
    """计算 (x^y) % m"""
    return pow(x, y, m)

def extended_gcd(a, b):
    """扩展欧几里得算法"""
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = extended_gcd(b % a, a)
        return (g, y - (b // a) * x, x)

def mod_inverse(a, m):
    """计算模反元素，即找到b使得(a*b) % m == 1"""
    g, x, y = extended_gcd(a, m)
    if g != 1:
        if a % m == 0:
            return 0 
        else:
            raise Exception(f'模反元素不存在: gcd({a}, {m}) = {g}')
    else:
        return x % m
    
class Montgomery_PCS:
    def __init__(self, MOD):
        self.MOD = MOD
        self.R = 1 << MOD.bit_length()
        self.INV_R = mod_inverse(self.R, MOD)
        _, inv, _ = extended_gcd(MOD, self.R)
        self.MOD_INV = (self.R - (inv % self.R)) % self.R
    
    def convert_to_mont(self, x):
        """将普通数转换为蒙哥马利形式"""
        return (x * self.R) % self.MOD
    
    def convert_from_mont(self, x):
        """将蒙哥马利形式转换为普通数"""
        return (x * self.INV_R) % self.MOD
    
    def mul(self, a, b):
        """蒙哥马利模乘 [a * b * (R^-1)] mod m"""
        multiply = a * b
        temp = (multiply * self.MOD_INV) & (self.R - 1)
        res = (multiply + temp * self.MOD) >> self.MOD.bit_length()
        return res % self.MOD
    
    def add(self, a, b):
        """蒙哥马利域中的加法 a + b mod m"""
        return (a + b) % self.MOD
    
    def sub(self, a, b):
        """蒙哥马利域中的减法 a - b mod m"""
        return (a - b) % self.MOD

def point_add_mont_jacobian(P, Q, a_mont, mont):
    """雅可比坐标系下使用蒙哥马利模乘的点加法 P + Q"""
    if P == INF:
        return Q
    if Q == INF:
        return P
    
    X1, Y1, Z1 = P
    X2, Y2, Z2 = Q
    Z1Z1 = mont.mul(Z1, Z1)
    Z2Z2 = mont.mul(Z2, Z2)
    U1 = mont.mul(X1, Z2Z2)
    U2 = mont.mul(X2, Z1Z1)
    Z2_cubed = mont.mul(Z2, Z2Z2)
    Z1_cubed = mont.mul(Z1, Z1Z1)
    S1 = mont.mul(Y1, Z2_cubed)
    S2 = mont.mul(Y2, Z1_cubed)
    
    if U1 == U2:
        if S1 == mont.sub(0, S2):
            return INF
    if U1 == U2 and S1 == S2:
        return point_double_mont_jacobian(P, a_mont, mont)
    
    H = mont.sub(U2, U1)
    R = mont.sub(S2, S1)
    H2 = mont.mul(H, H)
    H3 = mont.mul(H, H2)
    V = mont.mul(U1, H2)
    X3 = mont.sub(mont.sub(mont.mul(R, R), H3), mont.mul(V, mont.convert_to_mont(2)))
    Y3 = mont.sub(mont.mul(R, mont.sub(V, X3)), mont.mul(S1, H3))
    Z3 = mont.mul(mont.mul(Z1, Z2), H)
    return (X3, Y3, Z3)

def point_double_mont_jacobian(P, a_mont, mont):
    """雅可比坐标系下使用蒙哥马利模乘的点倍乘 2P"""
    if P == INF:
        return INF
    
    X, Y, Z = P

    if Y == 0:
        return INF
    
    Y2 = mont.mul(Y, Y)
    Y4 = mont.mul(Y2, Y2)
    four = mont.convert_to_mont(4)
    S = mont.mul(four, mont.mul(X, Y2))
    three = mont.convert_to_mont(3)
    X_squared = mont.mul(X, X)
    three_X_squared = mont.mul(three, X_squared)
    Z_squared = mont.mul(Z, Z)
    Z_fourth = mont.mul(Z_squared, Z_squared)
    a_Z_fourth = mont.mul(a_mont, Z_fourth)
    
    M = mont.add(three_X_squared, a_Z_fourth)
    
    X3 = mont.sub(mont.mul(M, M), mont.mul(S, mont.convert_to_mont(2)))
    Y3 = mont.sub(mont.mul(M, mont.sub(S, X3)), mont.mul(mont.convert_to_mont(8), Y4))
    Z3 = mont.mul(mont.convert_to_mont(2), mont.mul(Y, Z))
    return (X3, Y3, Z3)

def point_mul_mont_jacobian(k, P, a_mont, mont):
    """雅可比坐标系下使用蒙哥马利模乘的点乘法 kP"""
    if k == 0:
        return INF
    result = INF
    addend = P
    while k > 0:
        if k & 1:
            result = point_add_mont_jacobian(result, addend, a_mont, mont)
        addend = point_double_mont_jacobian(addend, a_mont, mont)
        k >>= 1
    return result

def mont_jacobian_to_affine(P, mont):
    """将蒙哥马利雅可比坐标转换为普通仿射坐标"""
    if P == INF:
        return None
    
    X, Y, Z = P
    Z_normal = mont.convert_from_mont(Z)
    Z_inv = mod_inverse(Z_normal, mont.MOD)
    Z2_inv = (Z_inv * Z_inv) % mont.MOD
    Z3_inv = (Z2_inv * Z_inv) % mont.MOD
    x = (mont.convert_from_mont(X) * Z2_inv) % mont.MOD
    y = (mont.convert_from_mont(Y) * Z3_inv) % mont.MOD
    return (x, y)

In [ ]:
import os

def sha256_mod_q(m):
    """计算SHA256(m) mod q"""
    if isinstance(m, str):
        m = m.encode('utf-8').hex().upper()
    Sha256 = SHA2_256()
    h = Sha256.sha256(m)
    return h % n

def bytes_to_int(b):
    """将字节转换为整数"""
    return int.from_bytes(b, 'big')

def int_to_bytes(i):
    """将整数转换为字节"""
    return i.to_bytes((i.bit_length() + 7) // 8, 'big')

def generate_keypair(x:int = None):
    """生成ECDSA密钥对"""
    mont_p = Montgomery_PCS(p)
    a_mont = mont_p.convert_to_mont(a)
    Gx_mont = mont_p.convert_to_mont(Gx)
    Gy_mont = mont_p.convert_to_mont(Gy)
    G_jacobian = (Gx_mont, Gy_mont, mont_p.convert_to_mont(1))
    if x == None :
        x = bytes_to_int(os.urandom(32)) % n
        while x == 0:
            x = bytes_to_int(os.urandom(32)) % n
    else:
        x = x % n
    Y_jacobian = point_mul_mont_jacobian(x, G_jacobian, a_mont, mont_p)
    Y_affine = mont_jacobian_to_affine(Y_jacobian, mont_p)
    return x, Y_affine

def sign(private_key, message):  # (..., k:int = None)
    """ECDSA签名算法"""
    mont_p = Montgomery_PCS(p)
    a_mont = mont_p.convert_to_mont(a)
    Gx_mont = mont_p.convert_to_mont(Gx)
    Gy_mont = mont_p.convert_to_mont(Gy)
    G_jacobian = (Gx_mont, Gy_mont, mont_p.convert_to_mont(1))
    h = sha256_mod_q(message)
    max_attempts = 1000
    for _ in range(max_attempts):
        # if k == None:
            # k = bytes_to_int(os.urandom(32)) % (n-1) + 1
        k = bytes_to_int(os.urandom(32)) % (n-1) + 1
        kG_jacobian = point_mul_mont_jacobian(k, G_jacobian, a_mont, mont_p)
        kG_affine = mont_jacobian_to_affine(kG_jacobian, mont_p)
        if kG_affine is None:
            continue
        x, y = kG_affine
        r = x % n
        if r == 0:
            continue
        k_inv = mod_inverse(k, n)
        s = (k_inv * (h + private_key * r)) % n
        if s == 0:
            continue
        return (r, s)
    raise Exception("无法生成有效的ECDSA签名")

def verify(public_key, message, signature):
    """ECDSA验证算法"""
    r, s = signature
    if r < 1 or r >= n or s < 1 or s >= n:
        return False
    mont_p = Montgomery_PCS(p)
    a_mont = mont_p.convert_to_mont(a)
    Gx_mont = mont_p.convert_to_mont(Gx)
    Gy_mont = mont_p.convert_to_mont(Gy)
    G_jacobian = (Gx_mont, Gy_mont, mont_p.convert_to_mont(1))
    Yx, Yy = public_key
    Yx_mont = mont_p.convert_to_mont(Yx)
    Yy_mont = mont_p.convert_to_mont(Yy)
    Y_jacobian = (Yx_mont, Yy_mont, mont_p.convert_to_mont(1))
    h = sha256_mod_q(message)
    w = mod_inverse(s, n)
    u1 = (h * w) % n
    u2 = (r * w) % n
    u1G_jacobian = point_mul_mont_jacobian(u1, G_jacobian, a_mont, mont_p)
    u2Y_jacobian = point_mul_mont_jacobian(u2, Y_jacobian, a_mont, mont_p)
    point_sum_jacobian = point_add_mont_jacobian(u1G_jacobian, u2Y_jacobian, a_mont, mont_p)
    point_sum_affine = mont_jacobian_to_affine(point_sum_jacobian, mont_p)
    if point_sum_affine is None:
        return False
    x, y = point_sum_affine
    v = x % n
    return v == r

In [50]:
private = random.randint(1, n - 1)

In [66]:
private_key, public_key = generate_keypair(private)
print("私钥:", hex(private_key))
print("公钥:", (hex(public_key[0]), hex(public_key[1])))
message = "Hello, ECDSA!"
signature = sign(private_key, message)
print("签名:", (hex(signature[0]), hex(signature[1])))
valid = verify(public_key, message, signature)
print("签名验证结果:", valid)

私钥: 0xe4e9de86cc373e50c842c1c6ba346789c313724ea8773c259321ae1dd621d739
公钥: ('0xc3ce81dd438f83210b15977eaa336afbe321cea9a5225525bf225844576b017b', '0xc61eb7c8d3f86fcab2730852751874751c92553715a58e4585c8f32dad4a4f6d')
签名: ('0xa6af23f21d2dd1cc0e26c66238fef262562dba3bb66a6d9e9ae314136b2bba8b', '0x60e39f9b587fce3bfc5cfdc859b41ecfd77209765d8e64ffca4e20dd0b739af6')
签名验证结果: True


## 2.2 标准库实现 ECDSA

In [ ]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

def generate_keypair():
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    public_key = private_key.public_key()
    return private_key, public_key

def sign_message(private_key, message: bytes) -> bytes:
    signature = private_key.sign(
        message,
        ec.ECDSA(hashes.SHA256())  
    )
    return signature

def verify_signature(public_key, message: bytes, signature: bytes) -> bool:
    try:
        public_key.verify(
            signature,
            message,
            ec.ECDSA(hashes.SHA256())
        )
        return True
    except Exception:
        return False

if __name__ == "__main__":
    private_key, public_key = generate_keypair()
    message = b"Hello, ECDSA!"
    signature = sign_message(private_key, message)  #  使用 ASN.1 DER 编码
    print(f"签名: {signature.hex()}")
    valid = verify_signature(public_key, message, signature)
    print(f"验证结果: {valid}")
    public_key_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    print(f"\n公钥 (PEM格式):\n{public_key_pem.decode('utf-8')}")

签名: 304602210087cd863dfb7299409d1ec8078463a89b9e2ebfeb6740046688ef5316896fbae1022100a8fd7d8d2d591ee97c777d4d5fed92a02f3e63f19c12908a6e6622d283fa2e7b
验证结果: True

公钥 (PEM格式):
-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEDEpuwtRFTuayHmloiJ0mNKACYSex
1Kj4eCRHN1lz4Syl8d6ue3KJTHrUVtsjIaitghfXhbeGiQUPccNosCc30Q==
-----END PUBLIC KEY-----

